Instructions:

-Open file in Jupyter notebook or Jupyter lab

-Enter number of stocks to analyze and dates of interest (start/end).

-Enter ticker symbols.

-Enter appropriate benchmark. Select S&P 500, Russell 3000, Nasdaq or enter your own index symbol.

-Enter the appropriate risk free rate to use.

-Program will scrape price data from Yahoo, convert price data to returns, run regression, interpret output and write data to file as a multi-tabbed excel file with each stock's regression in one tab. 

In [1]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import random
import statsmodels.api as sm
from statsmodels import regression
import matplotlib.pyplot as plt
from datetime import date

# Returns intercept, first coefficient, regression summary. ref: https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLSResults.summary2.html, https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html
def linreg(x,y):
    
    x1 = sm.add_constant(x)
    model = regression.linear_model.OLS(y,x1).fit()
    
#   Returns the beta, alpha and model summary
    print("0 coefficient: ", model.params[0])
    print("1 coefficient: ", model.params[1])
    print("summary2: ", model.summary2())
    print("summary1?: ", model.summary)
    return model.params[0], model.params[1], model.summary2()

# Menu of benchmark options
def get_benchmark():
    
    benchmark = input('Which benchmark ticker would you like to use? \n1 for S&P500, 2 for Russell 3000, 3 for NASDAQ, or enter your own\n')
    
    if benchmark == '1':
        benchmark = '^GSPC'
        print('You chose S&P500')
        return benchmark
    elif benchmark == '2':
        benchmark = '^RUA'
        print('You chose Russell 3000')
        return benchmark
    elif benchmark == '3':
        benchmark = '^IXIC'
        print('You chose NASDAQ')
        return benchmark
    else:
        return benchmark
def get_start_year():
    start_year = input('What year do you want to start on?\nFormat: MM/DD/YYYY\n ')
    return start_year

def get_end_year():
    end_year = input('What year do you want to end on?\nFormat: MM/DD/YYYY\n1 for today\n')
    if end_year == '1':
        today = date.today()
        end_year = today.strftime("%m/%d/%y")
        print('Today\'s date: ', end_year)
        return end_year
    else:
        return end_year

def betas():
    
    # Gather user input
    benchmark = get_benchmark()
    
    rf = float(input('What would you like to use for risk free rate? (Enter as a decimal. Ex: 2% would be 0.02)\n')) 
    rf_daily = np.power(1+rf,(1/365)) - 1
    print('Daily risk free rate:', rf_daily)
    #if you want to use a treasury bond, uncomment this code
#     rf = input('What would you like to use for risk free rate?\n IRX: 13 Week \n FVX: 5 Year \nTNX: 10 Year\n') 
#     rf = '^'+rf

    stocks = [benchmark]
    
    num_stocks = int(input('How many companies do you want to look at?\n'))    
    # Gets ticker symbols for the stocks
    while num_stocks != 0:
        s = input('please enter ticker symbol for one stock:\n')
        stocks.append(s)
        num_stocks -= 1
        
    start_year = get_start_year()
    start_year_name = start_year.replace('/','-') # For file name
    
    end_year = get_end_year()
    end_year_name = end_year.replace('/','-') # For file name
    
    # Scrape stock returns from Yahoo Finance 
    data = web.DataReader(stocks, data_source='yahoo', start = start_year, 
                       end = end_year)['Adj Close']
    
    # Get monthly returns if you do not want to use daily returns
#     data = data.groupby(pd.TimeGrouper(freq='MS'))[stocks].mean()
        
    # Create and name an excel notebook
    # Create Pandas Excel writer object w/ engine xlsxwriter
    writer = pd.ExcelWriter('{} capm {} to {}.xlsx'.format(benchmark, start_year_name, end_year_name), engine='xlsxwriter')
    workbook  = writer.book
    
#     data[benchmark] = data[benchmark]

    for stock in stocks[1:]:
        
        data[stock] = data[stock]
        
        # Calculate returns for the market and the stock
        # Subtract risk free rate from each stock to get stock premium
        returns = data[[benchmark, stock]].pct_change() - rf_daily
        returns = returns.dropna()
        print(returns)
        
        # Create X and y for the regression
        y = returns[stock]
        X = returns[benchmark]
        
        # Returns alpha, beta, and summary of regression
        a, b, model = linreg(X,y)
        
        # Create dataframe for 1st part of summary
        df = model.tables[0]
        print('First part of summary:\n', df)
        
        # Create dataframe for 2nd part of summary
        df1 = model.tables[1]
        print('Second part of summary:\n', df1)
        
        # Combine two parts of regression summary
        table = df.append(df1)
        
        # Write dataframe to Excel
        table.to_excel(writer, sheet_name='{}'.format(stock)) 
        
        # Create plot of security characteristic line
        plt.figure(figsize = (20,10))
        plt.scatter(X,y, alpha=0.3)
        p = np.linspace(X.min(),X.max(), 100)
        y1 = b*p + a #b = beta and a = alpha
        plt.plot(p, y1, 'r', alpha=.9) 
        plt.grid(True, which ='both')
        plt.axhline(y=0, color = 'k')
        plt.axvline(x=0, color = 'k')
        plt.title('{} Security Characteristic Line'.format(stock))
        plt.xlabel('Market Return')
        plt.ylabel('{} return'.format(stock))
        plt.savefig('{} {} to {}.png'.format(stock, start_year_name, end_year_name))
        
        # Save plot to a spreadsheet
        worksheet = writer.sheets['{}'.format(stock)]
        worksheet.insert_image('A15', '{} {} to {}.png'.format(stock, start_year_name, end_year_name))
        
        # Clear plot to make unique plot for each respective stock
        plt.clf()
        
    # Save and close Excel writer
    writer.save()
    writer.close()
    
    print('\nExported to Excel file and images.\n')

# Run the program
betas()

#current rf: .012

Which benchmark ticker would you like to use? 
1 for S&P500, 2 for Russell 3000, 3 for NASDAQ, or enter your own
 1


You chose S&P500


What would you like to use for risk free rate? (Enter as a decimal. Ex: 2% would be 0.02)
 .012


Daily risk free rate: 3.2681550099500356e-05


How many companies do you want to look at?
 2
please enter ticker symbol for one stock:
 GOOG
please enter ticker symbol for one stock:
 SIVB
What year do you want to start on?
Format: MM/DD/YYYY
  08/19/2004
What year do you want to end on?
Format: MM/DD/YYYY
1 for today
 1


Today's date:  03/04/20
Symbols        ^GSPC      GOOG
Date                          
2004-08-20  0.006492  0.079397
2004-08-23 -0.002464  0.010031
2004-08-24  0.000433 -0.041440
2004-08-25  0.007968  0.010743
2004-08-26  0.000085  0.017986
...              ...       ...
2020-02-27 -0.044196 -0.053931
2020-02-28 -0.008271  0.016082
2020-03-02  0.046007  0.037135
2020-03-03 -0.028141 -0.034386
2020-03-04  0.028132  0.020550

[3911 rows x 2 columns]
0 coefficient:  0.0006999134547691045
1 coefficient:  0.9697196502570715
summary2:                    Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.350      
Dependent Variable: GOOG             AIC:                -21628.9329
Date:               2020-03-04 13:16 BIC:                -21616.3898
No. Observations:   3911             Log-Likelihood:     10816.     
Df Model:           1                F-statistic:        2108.      
Df Residuals:       3909             Prob (F-statistic): 0.00       


C:\Users\Hannah\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Symbols        ^GSPC      SIVB
Date                          
2004-08-20  0.006492  0.013169
2004-08-23 -0.002464 -0.005733
2004-08-24  0.000433 -0.008223
2004-08-25  0.007968  0.007950
2004-08-26  0.000085  0.011983
...              ...       ...
2020-02-27 -0.044196 -0.057679
2020-02-28 -0.008271 -0.015219
2020-03-02  0.046007  0.041234
2020-03-03 -0.028141 -0.083631
2020-03-04  0.028132  0.007569

[3911 rows x 2 columns]
0 coefficient:  0.0002805974801623034
1 coefficient:  1.3756527979927116
summary2:                    Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.435      
Dependent Variable: SIVB             AIC:                -20295.9594
Date:               2020-03-04 13:16 BIC:                -20283.4163
No. Observations:   3911             Log-Likelihood:     10150.     
Df Model:           1                F-statistic:        3016.      
Df Residuals:       3909             Prob (F-statistic): 0.00       
R-squared:          0.43

In [2]:
# from datetime import date
# date.today()

datetime.date(2020, 3, 4)